In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [ ]:
image_directory = 'movie_dataset_multilabel/images/'
df = pd.read_csv('movie_dataset_multilabel/movie_metadata.csv')    
df = df.iloc[:2000]  # Using the same subset of data for memory reasons.

# Image transformations
transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])


In [ ]:
X_dataset = []
for i in tqdm(range(df.shape[0])):
    img = Image.open(image_directory + df['Id'][i] + '.jpg')
    img = transform(img)
    X_dataset.append(img)

X = torch.stack(X_dataset)
y = torch.tensor(np.array(df.drop(['Id', 'Genre'], axis=1)), dtype=torch.float32)

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20, test_size=0.3)

# Create DataLoaders
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
class MultiLabelNN(nn.Module):
    def __init__(self):
        super(MultiLabelNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, activation=F.relu)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=5, activation=F.relu)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=5, activation=F.relu)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=5, activation=F.relu)
        self.bn4 = nn.BatchNorm2d(64)
        self.fc1 = nn.Linear(64 * 10 * 10, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 25)  # Assuming 25 genres as in Keras example

    def forward(self, x):
        x = F.max_pool2d(self.bn1(self.conv1(x)), 2)
        x = F.dropout(x, 0.2)
        x = F.max_pool2d(self.bn2(self.conv2(x)), 2)
        x = F.dropout(x, 0.2)
        x = F.max_pool2d(self.bn3(self.conv3(x)), 2)
        x = F.dropout(x, 0.2)
        x = F.max_pool2d(self.bn4(self.conv4(x)), 2)
        x = F.dropout(x, 0.2)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.5)
        x = F.relu(self.fc2(x))
        x = F.dropout(x, 0.5)
        x = torch.sigmoid(self.fc3(x))
        return x

In [ ]:
model = MultiLabelNN()

In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters())
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        model.eval()
        with torch.no_grad():
            correct = 0
            total = 0
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                _, predicted = torch.max(outputs.data, 1)
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Accuracy: {100 * correct / total}%')
torch.save(model.state_dict(), 'movie_genre_classifier.pth')




In [ ]:
def evaluate_single_image(image_path, model, transform):
    img = Image.open(image_path)
    img = transform(img)
    img = img.unsqueeze(0)  # Add batch dimension

    model.eval()
    with torch.no_grad():
        outputs = model(img)
        probabilities = torch.sigmoid(outputs).squeeze(0)
        top_prob, top_catid = torch.topk(probabilities, 10)
        for i in range(top_prob.size(0)):
            print(df.columns[2:][top_catid[i]], top_prob[i].item())

evaluate_single_image('ddlj.jpg', model, transform)